In [1]:
%pip install pyannote.audio
# %pip install numpy==1.26 # 이미 있음?

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/898.7 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/898.7 kB ? eta -:--:--
   ---------------------------------------- 898.7/898.7 kB 3.1 MB/s  0:00:00
   ---------------------------------------- 0.0/824.2 kB ? eta -:--:--
   ------------------------- -------------- 524.3/824.2 kB 2.4 MB/s eta 0:00:01
   ---------------------------------------- 824.2/824.2 kB 2.1 MB/s  0:00:00
   ---------------------------------------- 0.0/983.0 kB ? eta -:--:--
   --------------------- ------------------ 524.3/983.0 kB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 983.0/983.0 kB 2.7 MB/s  0:00:00
   --------------

  DEPRECATION: Building 'antlr4-python3-runtime' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'antlr4-python3-runtime'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  DEPRECATION: Building 'docopt' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'docopt'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  DEPRECATION: Building 'julius' using the legacy setup.

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

HUGGING_FACE_TOKEN = os.getenv("HUGGING_FACE_TOKEN")

In [3]:
# instantiate the pipeline
from pyannote.audio import Pipeline

pipeline = Pipeline.from_pretrained(
  "pyannote/speaker-diarization-3.1",
  use_auth_token=HUGGING_FACE_TOKEN
)

c:\Users\USER\miniconda3\envs\GJ_class\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\USER\miniconda3\envs\GJ_class\Lib\site-packages\pyannote\audio\core\io.py:212: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  torchaudio.list_audio_backends()
c:\Users\USER\miniconda3\envs\GJ_class\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated file

In [4]:
import torch

# cuda가 사용 가능한 경우 cuda를 사용하도록 설정
if torch.cuda.is_available():
    pipeline.to(torch.device("cuda"))
    print('cuda is available')
else:
    print('cuda is not available')

cuda is available


In [5]:
# run the pipeline on an audio file
# diarization = pipeline("audio.wav")
diarization = pipeline("./audio/싼기타_비싼기타.mp3")

# dump the diarization output to disk using RTTM format
# 화자를 분리하기 위해서는 반드시 rttm 포맷으로 데이터를 변경해야한다.
with open("싼기타_비싼기타.rttm", "w", encoding='utf-8') as rttm:
    diarization.write_rttm(rttm)

c:\Users\USER\miniconda3\envs\GJ_class\Lib\site-packages\pyannote\audio\utils\reproducibility.py:74: ReproducibilityWarning: TensorFloat-32 (TF32) has been disabled as it might lead to reproducibility issues and lower accuracy.
It can be re-enabled by calling
   >>> import torch
   >>> torch.backends.cuda.matmul.allow_tf32 = True
   >>> torch.backends.cudnn.allow_tf32 = True
See https://github.com/pyannote/pyannote-audio/issues/1370 for more details.

  warnings.warn(
c:\Users\USER\miniconda3\envs\GJ_class\Lib\site-packages\torchaudio\_backend\utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  

In [6]:
# RTTM을 CSV로 변환
import pandas as pd
rttm_path = "./싼기타_비싼기타.rttm"

df_rttm = pd.read_csv(
    rttm_path,      # rttm 파일 경로
    sep=' ',        # 구분자는 띄어쓰기
    header=None,    # 헤더는 없음
    names=['type', 'file', 'chnl', 'start', 'duration', 'C1', 'C2', 'speaker_id', 'C3', 'C4'] 
)

display(df_rttm)

,type,file,chnl,start,duration,C1,C2,speaker_id,C3,C4
0,SPEAKER,싼기타_비싼기타,1,0.993,5.805,NaN,NaN,SPEAKER_01,NaN,NaN
1,SPEAKER,싼기타_비싼기타,1,7.405,3.983,NaN,NaN,SPEAKER_01,NaN,NaN
2,SPEAKER,싼기타_비싼기타,1,11.759,4.927,NaN,NaN,SPEAKER_01,NaN,NaN
3,SPEAKER,싼기타_비싼기타,1,17.210,10.665,NaN,NaN,SPEAKER_01,NaN,NaN
4,SPEAKER,싼기타_비싼기타,1,28.668,1.536,NaN,NaN,SPEAKER_01,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
83,SPEAKER,싼기타_비싼기타,1,414.481,2.970,NaN,NaN,SPEAKER_00,NaN,NaN
84,SPEAKER,싼기타_비싼기타,1,417.755,3.476,NaN,NaN,SPEAKER_01,NaN,NaN
85,SPEAKER,싼기타_비싼기타,1,423.644,0.776,NaN,NaN,SPEAKER_00,NaN,NaN
86,SPEAKER,싼기타_비싼기타,1,424.741,3.527,NaN,NaN,SPEAKER_00,NaN,NaN


In [7]:
# start + duration을 end로 변환
df_rttm['end'] = df_rttm['start'] + df_rttm['duration']

display(df_rttm)

,type,file,chnl,start,duration,C1,C2,speaker_id,C3,C4,end
0,SPEAKER,싼기타_비싼기타,1,0.993,5.805,NaN,NaN,SPEAKER_01,NaN,NaN,6.798
1,SPEAKER,싼기타_비싼기타,1,7.405,3.983,NaN,NaN,SPEAKER_01,NaN,NaN,11.388
2,SPEAKER,싼기타_비싼기타,1,11.759,4.927,NaN,NaN,SPEAKER_01,NaN,NaN,16.686
3,SPEAKER,싼기타_비싼기타,1,17.210,10.665,NaN,NaN,SPEAKER_01,NaN,NaN,27.875
4,SPEAKER,싼기타_비싼기타,1,28.668,1.536,NaN,NaN,SPEAKER_01,NaN,NaN,30.204
...,...,...,...,...,...,...,...,...,...,...,...
83,SPEAKER,싼기타_비싼기타,1,414.481,2.970,NaN,NaN,SPEAKER_00,NaN,NaN,417.451
84,SPEAKER,싼기타_비싼기타,1,417.755,3.476,NaN,NaN,SPEAKER_01,NaN,NaN,421.231
85,SPEAKER,싼기타_비싼기타,1,423.644,0.776,NaN,NaN,SPEAKER_00,NaN,NaN,424.420
86,SPEAKER,싼기타_비싼기타,1,424.741,3.527,NaN,NaN,SPEAKER_00,NaN,NaN,428.268


In [8]:
df_rttm["number"] = None    # number 열 만들고 None으로 초기화
df_rttm.at[0, "number"] = 0

display(df_rttm)

,type,file,chnl,start,duration,C1,C2,speaker_id,C3,C4,end,number
0,SPEAKER,싼기타_비싼기타,1,0.993,5.805,NaN,NaN,SPEAKER_01,NaN,NaN,6.798,0
1,SPEAKER,싼기타_비싼기타,1,7.405,3.983,NaN,NaN,SPEAKER_01,NaN,NaN,11.388,None
2,SPEAKER,싼기타_비싼기타,1,11.759,4.927,NaN,NaN,SPEAKER_01,NaN,NaN,16.686,None
3,SPEAKER,싼기타_비싼기타,1,17.210,10.665,NaN,NaN,SPEAKER_01,NaN,NaN,27.875,None
4,SPEAKER,싼기타_비싼기타,1,28.668,1.536,NaN,NaN,SPEAKER_01,NaN,NaN,30.204,None
...,...,...,...,...,...,...,...,...,...,...,...,...
83,SPEAKER,싼기타_비싼기타,1,414.481,2.970,NaN,NaN,SPEAKER_00,NaN,NaN,417.451,None
84,SPEAKER,싼기타_비싼기타,1,417.755,3.476,NaN,NaN,SPEAKER_01,NaN,NaN,421.231,None
85,SPEAKER,싼기타_비싼기타,1,423.644,0.776,NaN,NaN,SPEAKER_00,NaN,NaN,424.420,None
86,SPEAKER,싼기타_비싼기타,1,424.741,3.527,NaN,NaN,SPEAKER_00,NaN,NaN,428.268,None


In [9]:
for i in range(1, len(df_rttm)):
    if df_rttm.at[i, "speaker_id"] != df_rttm.at[i-1, "speaker_id"]:
        df_rttm.at[i, "number"] = df_rttm.at[i-1, "number"] + 1
    else:
        df_rttm.at[i, "number"] = df_rttm.at[i-1, "number"]

display(df_rttm.head(10))

,type,file,chnl,start,duration,C1,C2,speaker_id,C3,C4,end,number
0,SPEAKER,싼기타_비싼기타,1,0.993,5.805,NaN,NaN,SPEAKER_01,NaN,NaN,6.798,0
1,SPEAKER,싼기타_비싼기타,1,7.405,3.983,NaN,NaN,SPEAKER_01,NaN,NaN,11.388,0
2,SPEAKER,싼기타_비싼기타,1,11.759,4.927,NaN,NaN,SPEAKER_01,NaN,NaN,16.686,0
3,SPEAKER,싼기타_비싼기타,1,17.210,10.665,NaN,NaN,SPEAKER_01,NaN,NaN,27.875,0
4,SPEAKER,싼기타_비싼기타,1,28.668,1.536,NaN,NaN,SPEAKER_01,NaN,NaN,30.204,0
5,SPEAKER,싼기타_비싼기타,1,32.414,0.759,NaN,NaN,SPEAKER_00,NaN,NaN,33.173,1
6,SPEAKER,싼기타_비싼기타,1,33.545,3.561,NaN,NaN,SPEAKER_00,NaN,NaN,37.106,1
7,SPEAKER,싼기타_비싼기타,1,37.628,3.763,NaN,NaN,SPEAKER_00,NaN,NaN,41.391,1
8,SPEAKER,싼기타_비싼기타,1,41.611,1.097,NaN,NaN,SPEAKER_00,NaN,NaN,42.708,1
9,SPEAKER,싼기타_비싼기타,1,41.645,0.810,NaN,NaN,SPEAKER_01,NaN,NaN,42.455,2


In [10]:
df_rttm_grouped = df_rttm.groupby("number").agg(
    start=pd.NamedAgg(column='start', aggfunc='min'),
    end=pd.NamedAgg(column='end', aggfunc='max'),
    speaker_id=pd.NamedAgg(column='speaker_id', aggfunc='first')
)

display(df_rttm_grouped)

,start,end,speaker_id
number,,,
0,0.993,30.204,SPEAKER_01
1,32.414,42.708,SPEAKER_00
2,41.645,44.024,SPEAKER_01
3,45.813,67.109,SPEAKER_00
4,67.227,82.786,SPEAKER_01
5,84.659,102.564,SPEAKER_00
6,103.492,117.532,SPEAKER_01
7,119.759,138.676,SPEAKER_00
8,139.351,168.967,SPEAKER_01


In [11]:
df_rttm_grouped["duration"] = df_rttm_grouped["end"] - df_rttm_grouped["start"]
df_rttm_grouped = df_rttm_grouped.reset_index(drop=True)
display(df_rttm_grouped)

,start,end,speaker_id,duration
0,0.993,30.204,SPEAKER_01,29.211
1,32.414,42.708,SPEAKER_00,10.294
2,41.645,44.024,SPEAKER_01,2.379
3,45.813,67.109,SPEAKER_00,21.296
4,67.227,82.786,SPEAKER_01,15.559
5,84.659,102.564,SPEAKER_00,17.905
6,103.492,117.532,SPEAKER_01,14.040
7,119.759,138.676,SPEAKER_00,18.917
8,139.351,168.967,SPEAKER_01,29.616
9,170.907,192.321,SPEAKER_00,21.414


In [12]:
df_rttm_grouped.to_csv(
    "싼기타_비싼기타_rttm.csv",
    sep=',',
    index=False
)